In [ ]:
!pip uninstall torch
!pip install torch==1.8.1 torchvision==0.9.1 torchaudio==0.8.1
import torch
print(torch.__version__)
import torchvision
from torchvision import datasets, transforms

Found existing installation: torch 1.11.0+cu113
Uninstalling torch-1.11.0+cu113:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.7/dist-packages/caffe2/*
    /usr/local/lib/python3.7/dist-packages/torch-1.11.0+cu113.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torch/*
Proceed (y/n)? n
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 804.1 MB 2.6 kB/s 
     |████████████████████████████████| 17.4 MB 799 kB/s 
     |████████████████████████████████| 1.9 MB 41.2 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfull

In [ ]:
def prepare_dataloader(num_workers=8,
                       train_batch_size=128,
                       eval_batch_size=256):

    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406),
                             std=(0.229, 0.224, 0.225)),
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406),
                             std=(0.229, 0.224, 0.225)),
    ])

    train_set = torchvision.datasets.CIFAR10(root="data",
                                             train=True,
                                             download=True,
                                             transform=train_transform)
    # We will use test set for validation and test in this project.
    # Do not use test set for validation in practice!
    test_set = torchvision.datasets.CIFAR10(root="data",
                                            train=False,
                                            download=True,
                                            transform=test_transform)

    train_sampler = torch.utils.data.RandomSampler(train_set)
    test_sampler = torch.utils.data.SequentialSampler(test_set)

    train_loader = torch.utils.data.DataLoader(dataset=train_set,
                                               batch_size=train_batch_size,
                                               sampler=train_sampler,
                                               num_workers=num_workers)

    test_loader = torch.utils.data.DataLoader(dataset=test_set,
                                              batch_size=eval_batch_size,
                                              sampler=test_sampler,
                                              num_workers=num_workers)

    return train_loader, test_loader

In [ ]:
import os
import random
import torch
import time
import numpy as np

def set_random_seeds(random_seed=0):
    torch.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

def measure_inference_latency(model,
                              device,
                              input_size=(1, 3, 32, 32),
                              num_samples=100,
                              num_warmups=10):
    model.to(device)
    model.eval()

    x = torch.rand(size=input_size).to(device)

    with torch.no_grad():
        for _ in range(num_warmups):
            _ = model(x)

    if torch.cuda.is_available():
        torch.cuda.synchronize()

    with torch.no_grad():
        start_time = time.time()
        for _ in range(num_samples):
            _ = model(x)

            # torch.cuda.synchronize()
        end_time = time.time()
    elapsed_time = end_time - start_time
    elapsed_time_ave = elapsed_time / num_samples

    return elapsed_time_ave

def run_benchmark(model_file, img_loader):
    elapsed = 0
    model = torch.jit.load(model_file)
    model.eval()
    num_batches = 5
    # Run the scripted model on a few batches of images
    for i, (images, target) in enumerate(img_loader):
        if i < num_batches:
            start = time.time()
            output = model(images)
            end = time.time()
            elapsed = elapsed + (end-start)
        else:
            break
    num_images = images.size()[0] * num_batches

    print('Elapsed time: %3.0f ms' % (elapsed/num_images*1000))
    return elapsed


def model_equivalence(model_1, model_2, device, rtol=1e-05, atol=1e-08, num_tests=100, input_size=(1, 3, 32, 32)):
    model_1.to(device)
    model_2.to(device)

    for _ in range(num_tests):
        x = torch.rand(size=input_size).to(device)
        y1 = model_1(x).detach().cpu().numpy()
        y2 = model_2(x).detach().cpu().numpy()
        if np.allclose(a=y1, b=y2, rtol=rtol, atol=atol, equal_nan=False) == False:
            print("Model equivalence test sample failed: ")
            print(y1)
            print(y2)
            return False

    return True

def print_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    print("%.2f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

In [ ]:
# Modified from
# https://github.com/pytorch/vision/blob/release/0.8.0/torchvision/models/resnet.py

import torch
from torch import Tensor
import torch.nn as nn
from torch.hub import load_state_dict_from_url
from typing import Type, Any, Callable, Union, List, Optional

__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152', 'resnext50_32x4d', 'resnext101_32x8d',
           'wide_resnet50_2', 'wide_resnet101_2']

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
    'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}


def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
            self,
            inplanes: int,
            planes: int,
            stride: int = 1,
            downsample: Optional[nn.Module] = None,
            groups: int = 1,
            base_width: int = 64,
            dilation: int = 1,
            norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        # Rename relu to relu1
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride
        self.skip_add = nn.quantized.FloatFunctional()
        # Remember to use two independent ReLU for layer fusion.
        self.relu2 = nn.ReLU(inplace=True)

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        # Use FloatFunctional for addition for quantization compatibility
        # out += identity
        out = self.skip_add.add(identity, out)
        out = self.relu2(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
            self,
            inplanes: int,
            planes: int,
            stride: int = 1,
            downsample: Optional[nn.Module] = None,
            groups: int = 1,
            base_width: int = 64,
            dilation: int = 1,
            norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu1 = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.skip_add = nn.quantized.FloatFunctional()
        self.relu2 = nn.ReLU(inplace=True)
        self.relu3 = nn.ReLU(inplace=True)

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        # out += identity
        out = self.skip_add.add(identity, out)
        out = self.relu3(out)

        return out


class ResNet(nn.Module):

    def __init__(
            self,
            block: Type[Union[BasicBlock, Bottleneck]],
            layers: List[int],
            num_classes: int = 1000,
            zero_init_residual: bool = False,
            groups: int = 1,
            width_per_group: int = 64,
            replace_stride_with_dilation: Optional[List[bool]] = None,
            norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def _resnet(
        arch: str,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        pretrained: bool,
        progress: bool,
        **kwargs: Any
) -> ResNet:
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)


def resnet34(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-34 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet34', BasicBlock, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)


def resnet50(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)


def resnet101(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-101 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet101', Bottleneck, [3, 4, 23, 3], pretrained, progress,
                   **kwargs)


def resnet152(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-152 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet152', Bottleneck, [3, 8, 36, 3], pretrained, progress,
                   **kwargs)


def resnext50_32x4d(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNeXt-50 32x4d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 4
    return _resnet('resnext50_32x4d', Bottleneck, [3, 4, 6, 3],
                   pretrained, progress, **kwargs)


def resnext101_32x8d(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNeXt-101 32x8d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 8
    return _resnet('resnext101_32x8d', Bottleneck, [3, 4, 23, 3],
                   pretrained, progress, **kwargs)


def wide_resnet50_2(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""Wide ResNet-50-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_.

    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _resnet('wide_resnet50_2', Bottleneck, [3, 4, 6, 3],
                   pretrained, progress, **kwargs)


def wide_resnet101_2(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""Wide ResNet-101-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_.

    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _resnet('wide_resnet101_2', Bottleneck, [3, 4, 23, 3],
                   pretrained, progress, **kwargs)

In [ ]:
import torch
import torch.nn as nn

class QuantizedResNet(nn.Module):
    def __init__(self, model_fp32):
        super(QuantizedResNet, self).__init__()
        # QuantStub converts tensors from floating point to quantized.
        # This will only be used for inputs.
        self.quant = torch.quantization.QuantStub()
        # DeQuantStub converts tensors from quantized to floating point.
        # This will only be used for outputs.
        self.dequant = torch.quantization.DeQuantStub()
        # FP32 model
        self.model_fp32 = model_fp32

    def forward(self, x):
        # manually specify where tensors will be converted from floating
        # point to quantized in the quantized model
        x = self.quant(x)
        x = self.model_fp32(x)
        # manually specify where tensors will be converted from quantized
        # to floating point in the quantized model
        x = self.dequant(x)
        return x

In [ ]:
import os
import argparse

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

import copy

def evaluate_model(model, test_loader, device, criterion=None):
    model.eval()
    model.to(device)

    running_loss = 0
    running_corrects = 0

    for inputs, labels in test_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        if criterion is not None:
            loss = criterion(outputs, labels).item()
        else:
            loss = 0

        # statistics
        running_loss += loss * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    eval_loss = running_loss / len(test_loader.dataset)
    eval_accuracy = running_corrects / len(test_loader.dataset)

    return eval_loss, eval_accuracy

def train_model(model,
                train_loader,
                test_loader,
                device,
                learning_rate=1e-1,
                num_epochs=50):

    # The training configurations were not carefully selected.

    criterion = nn.CrossEntropyLoss()

    model.to(device)

    # It seems that SGD optimizer is better than Adam optimizer for ResNet18 training on CIFAR10.
    optimizer = optim.SGD(model.parameters(),
                          lr=learning_rate,
                          momentum=0.9,
                          weight_decay=1e-4)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=500)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,
                                                     milestones=[100, 150],
                                                     gamma=0.1,
                                                     last_epoch=-1)
    # optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

    # Evaluation
    model.eval()
    eval_loss, eval_accuracy = evaluate_model(model=model,
                                              test_loader=test_loader,
                                              device=device,
                                              criterion=criterion)
    print("Epoch: {:02d} Eval Loss: {:.3f} Eval Acc: {:.3f}".format(
        -1, eval_loss, eval_accuracy))

    for epoch in range(num_epochs):

        # Training
        model.train()

        running_loss = 0
        running_corrects = 0

        for inputs, labels in train_loader:

            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = running_corrects / len(train_loader.dataset)

        # Evaluation
        model.eval()
        eval_loss, eval_accuracy = evaluate_model(model=model,
                                                  test_loader=test_loader,
                                                  device=device,
                                                  criterion=criterion)

        # Set learning rate scheduler
        scheduler.step()

        print(
            "Epoch: {:03d} Train Loss: {:.3f} Train Acc: {:.3f} Eval Loss: {:.3f} Eval Acc: {:.3f}"
            .format(epoch, train_loss, train_accuracy, eval_loss,
                    eval_accuracy))

    return model

def calibrate_model(model, loader, device=torch.device("cpu:0")):
    model.to(device)
    model.eval()

    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        _ = model(inputs)

def save_model(model, model_dir, model_filename):
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_filepath = os.path.join(model_dir, model_filename)
    torch.save(model.state_dict(), model_filepath)

def load_model(model, model_filepath, device):
    model.load_state_dict(torch.load(model_filepath, map_location=device))

    return model

def save_torchscript_model(model, model_dir, model_filename):
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_filepath = os.path.join(model_dir, model_filename)
    torch.jit.save(torch.jit.script(model), model_filepath)


def load_torchscript_model(model_filepath, device):
    model = torch.jit.load(model_filepath, map_location=device)

    return model

def create_model(num_classes=10):
    # The number of channels in ResNet18 is divisible by 8.
    # This is required for fast GEMM integer matrix multiplication.
    # model = torchvision.models.resnet18(pretrained=False)
    model = resnet18(num_classes=num_classes, pretrained=False)
    #model = resnet50(num_classes=num_classes, pretrained=False)

    # We would use the pretrained ResNet18 as a feature extractor.
    # for param in model.parameters():
    #     param.requires_grad = False

    # Modify the last FC layer
    # num_features = model.fc.in_features
    # model.fc = nn.Linear(num_features, 10)

    return model

def gpu(tutorial_quantization, static_quantization, quantization_aware_training):
    random_seed = 0
    num_classes = 10
    cpu_device = torch.device("cpu:0")
    if torch.cuda.is_available():
        cuda_device = torch.device("cuda:0")
    else:
        assert AssertionError

    model_dir = "saved_models"
    model_filename = "resnet18_cifar10.pt"
    tutorial_quantized_model_filename = "resnet18_tutorial_quantized_imagenet.pt"
    static_jit_model_filename = "resnet18_static_jit_cifar10.pt"
    static_quantized_model_filename = "resnet18_static_quantized_cifar10.pt"
    qat_quantized_model_filename = "resnet18_qat_quantized_cifar10.pt"
    model_filepath = os.path.join(model_dir, model_filename)
    tutorial_quantized_model_filepath = os.path.join(model_dir, tutorial_quantized_model_filename)
    static_jit_model_filepath = os.path.join(model_dir, static_jit_model_filename)
    static_quantized_model_filepath = os.path.join(model_dir, static_quantized_model_filename)
    qat_quantized_model_filepath = os.path.join(model_dir, qat_quantized_model_filename)

    set_random_seeds(random_seed=random_seed)

    # argument parse and create log
    #args = parse_args()

    # Create an untrained model.
    model = create_model(num_classes=num_classes)

    train_loader, test_loader = prepare_dataloader(num_workers=8, train_batch_size=128, eval_batch_size=256)

    #if not os.path.isfile(model_filepath):
    # Train model.
    start = time.time()
    model = train_model(model=model, train_loader=train_loader, test_loader=test_loader, device=cuda_device, learning_rate=1e-1, num_epochs=50)
    end = time.time()
    print("Normal Training Time: ", end - start)

    # Save model.
    save_model(model=model, model_dir=model_dir, model_filename=model_filename)

    # Load a pretrained model.
    model = load_model(model=model, model_filepath=model_filepath, device=cuda_device)
    # Move the model to CPU since static quantization does not support CUDA currently.
    model.to(cpu_device)
    # Make a copy of the model for layer fusion
    fused_model = copy.deepcopy(model)

    model.eval()
    # The model has to be switched to evaluation mode before any layer fusion.
    # Otherwise the quantization will not work correctly.
    fused_model.eval()

    _, fp32_eval_accuracy = evaluate_model(model=model, test_loader=test_loader, device=cpu_device, criterion=None)

    if tutorial_quantization == True:
        print("*** Result of Tutorial Quantization ***")
        tutorial_model = torchvision.models.resnet18(pretrained=True)
        tutorial_quantized_model = torchvision.models.quantization.resnet18(pretrained=True, quantize=True)

        print("Resnet18 Model Size:")
        print_model_size(tutorial_model)
        print("Tutorial Quantized Model Size:")
        print_model_size(tutorial_quantized_model)

        save_model(model=tutorial_quantized_model, model_dir=model_dir, model_filename=tutorial_quantized_model_filename)

        # Load quantized model.
        # tutorial_quantized_jit_model = load_torchscript_model(model_filepath=tutorial_quantized_model_filepath,
        #                                                     device=cpu_device)

        tutorial_fp32_cpu_inference_latency = measure_inference_latency(model=tutorial_model, device=cpu_device,
                                                               input_size=(1, 3, 256, 256),
                                                               num_samples=100)
        tutorial_fp32_gpu_inference_latency = measure_inference_latency(model=tutorial_model, device=cuda_device,
                                                               input_size=(1, 3, 256, 256), num_samples=100)

        tutorial_int8_cpu_inference_latency = measure_inference_latency(model=tutorial_quantized_model, device=cpu_device,
                                                               input_size=(1, 3, 256, 256), num_samples=100)
        # tutorial_int8_jit_cpu_inference_latency = measure_inference_latency(model=tutorial_quantized_jit_model, device=cpu_device,
        #                                                            input_size=(1, 3, 256, 256), num_samples=100)

        
        print("FP32 CPU Inference Latency: {:.2f} ms / sample".format(tutorial_fp32_cpu_inference_latency * 1000))
        print("FP32 CUDA Inference Latency: {:.2f} ms / sample".format(tutorial_fp32_gpu_inference_latency * 1000))
        print("INT8 CPU Inference Latency: {:.2f} ms / sample".format(tutorial_int8_cpu_inference_latency * 1000))
        # print("INT8 JIT CPU Inference Latency: {:.2f} ms / sample".format(tutorial_int8_jit_cpu_inference_latency * 1000))

    if static_quantization == True:
        print("*** Result of Static Quantization ***")
        # Fuse the model in place rather manually.
        static_model = torch.quantization.fuse_modules(fused_model, [["conv1", "bn1", "relu"]], inplace=True)
        for module_name, module in static_model.named_children():
            if "layer" in module_name:
                for basic_block_name, basic_block in module.named_children():
                    torch.quantization.fuse_modules(basic_block, [["conv1", "bn1", "relu1"], ["conv2", "bn2", "relu2"], ["conv3", "bn3"]],
                                                    inplace=True)
                    for sub_block_name, sub_block in basic_block.named_children():
                        if sub_block_name == "downsample":
                            torch.quantization.fuse_modules(sub_block, [["0", "1"]], inplace=True)

        # Print FP32 model.
        #print(model)
        # Print fused model.
        #print(static_model)

        # Model and fused model should be equivalent.
        assert model_equivalence(model_1=model, model_2=static_model, device=cpu_device, rtol=1e-03, atol=1e-06,
                                 num_tests=100,
                                 input_size=(1, 3, 32, 32)), "Fused model is not equivalent to the original model!"

        # Prepare the model for static quantization. This inserts observers in
        # the model that will observe activation tensors during calibration.
        static_quantized_model = QuantizedResNet(model_fp32=static_model)
        # Using un-fused model will fail.
        # Because there is no quantized layer implementation for a single batch normalization layer.
        # static_quantized_model = QuantizedResNet18(model_fp32=model)
        # Select quantization schemes from
        # https://pytorch.org/docs/stable/quantization-support.html
        quantization_config = torch.quantization.get_default_qconfig("fbgemm")
        # Custom quantization configurations
        # quantization_config = torch.quantization.default_qconfig
        # quantization_config = torch.quantization.QConfig(activation=torch.quantization.MinMaxObserver.with_args(dtype=torch.quint8), weight=torch.quantization.MinMaxObserver.with_args(dtype=torch.qint8, qscheme=torch.per_tensor_symmetric))

        static_quantized_model.qconfig = quantization_config

        # Print quantization configurations
        print(static_quantized_model.qconfig)

        torch.quantization.prepare(static_quantized_model, inplace=True)

        # Use training data for calibration.
        calibrate_model(model=static_quantized_model, loader=train_loader, device=cpu_device)

        static_quantized_model = torch.quantization.convert(static_quantized_model, inplace=True)

        # Using high-level static quantization wrapper
        # The above steps, including torch.quantization.prepare, calibrate_model, and torch.quantization.convert, are also equivalent to
        # static_quantized_model = torch.quantization.quantize(model=static_quantized_model, run_fn=calibrate_model, run_args=[train_loader], mapping=None, inplace=False)

        static_quantized_model.eval()

        # Print quantized model.
        #print(static_quantized_model)

        print("FP32 Model Size:")
        print_model_size(model)
        # Save quantized model.
        save_torchscript_model(model=model, model_dir=model_dir,
                               model_filename=static_jit_model_filename)
        print("Static Quantized Model Size:")
        print_model_size(static_quantized_model)
        save_torchscript_model(model=static_quantized_model, model_dir=model_dir,
                               model_filename=static_quantized_model_filename)

        # Load quantized model.
        static_quantized_jit_model = load_torchscript_model(model_filepath=static_quantized_model_filepath,
                                                            device=cpu_device)

        _, static_int8_eval_accuracy = evaluate_model(model=static_quantized_jit_model, test_loader=test_loader,
                                               device=cpu_device,
                                               criterion=None)

        static_int8_cpu_inference_latency = measure_inference_latency(model=static_quantized_model, device=cpu_device,
                                                               input_size=(1, 3, 32, 32), num_samples=100)
        static_int8_jit_cpu_inference_latency = measure_inference_latency(model=static_quantized_jit_model, device=cpu_device,
                                                                   input_size=(1, 3, 32, 32), num_samples=100)

    if quantization_aware_training == True:
        print("*** Result of Quantization Aware Training ***")
        # The model has to be switched to training mode before any layer fusion.
        # Otherwise the quantization aware training will not work correctly.
        fused_model.train()

        # Fuse the model in place rather manually.
        qat_model = torch.quantization.fuse_modules(fused_model, [["conv1", "bn1"]], inplace=True)
        for module_name, module in qat_model.named_children():
            if "layer" in module_name:
                for basic_block_name, basic_block in module.named_children():
                    torch.quantization.fuse_modules(basic_block, [["conv1", "bn1", "relu1"], ["conv2", "bn2", "relu2"], ["conv3", "bn3"]],
                                                    inplace=True)
                    for sub_block_name, sub_block in basic_block.named_children():
                        if sub_block_name == "downsample":
                            torch.quantization.fuse_modules(sub_block, [["0", "1"]], inplace=True)

        # Print FP32 model.
        print(model)
        # Print fused model.
        print(qat_model)

        # equivalence will not pass if model is not eval mode
        qat_model.eval()

        # Model and fused model should be equivalent.
        assert model_equivalence(model_1=model, model_2=qat_model, device=cpu_device, rtol=1e-03, atol=1e-06,
                                 num_tests=100,
                                 input_size=(1, 3, 32, 32)), "Fused model is not equivalent to the original model!"

        # Prepare the model for static quantization. This inserts observers in
        # the model that will observe activation tensors during calibration.
        qat_quantized_model = QuantizedResNet(model_fp32=qat_model)
        # Using un-fused model will fail.
        # Because there is no quantized layer implementation for a single batch normalization layer.
        # qat_quantized_model = QuantizedResNet18(model_fp32=model)
        # Select quantization schemes from
        # https://pytorch.org/docs/stable/quantization-support.html
        quantization_config = torch.quantization.get_default_qconfig("fbgemm")
        # Custom quantization configurations
        # quantization_config = torch.quantization.default_qconfig
        # quantization_config = torch.quantization.QConfig(activation=torch.quantization.MinMaxObserver.with_args(dtype=torch.quint8), weight=torch.quantization.MinMaxObserver.with_args(dtype=torch.qint8, qscheme=torch.per_tensor_symmetric))

        qat_quantized_model.qconfig = quantization_config

        # Print quantization configurations
        print(qat_quantized_model.qconfig)

        # https://pytorch.org/docs/stable/_modules/torch/quantization/quantize.html#prepare_qat
        torch.quantization.prepare_qat(qat_quantized_model, inplace=True)
        # # Use training data for calibration.
        print("Training QAT Model...")
        
        qat_quantized_model.train()
        start = time.time()
        train_model(model=qat_quantized_model,
                    train_loader=train_loader,
                    test_loader=test_loader,
                    device=cuda_device,
                    learning_rate=1e-3,
                    num_epochs=25)
        end = time.time()
        print("QAT Training Time: ", end - start)

        qat_quantized_model.to(cpu_device)

        qat_quantized_model = torch.quantization.convert(qat_quantized_model, inplace=True)

        qat_quantized_model.eval()

        # Print quantized model.
        print(qat_quantized_model)

        print("Quantization Aware Training Model Size:")
        print_model_size(qat_quantized_model)
        # Save quantized model.
        save_torchscript_model(model=qat_quantized_model,
                               model_dir=model_dir,
                               model_filename=qat_quantized_model_filename)

        # Load quantized model.
        qat_quantized_jit_model = load_torchscript_model(
            model_filepath=qat_quantized_model_filepath, device=cpu_device)

        _, qat_int8_eval_accuracy = evaluate_model(model=qat_quantized_jit_model, test_loader=test_loader,
                                               device=cpu_device,
                                               criterion=None)

        qat_int8_cpu_inference_latency = measure_inference_latency(model=qat_quantized_model, device=cpu_device,
                                                               input_size=(1, 3, 32, 32), num_samples=100)
        qat_int8_jit_cpu_inference_latency = measure_inference_latency(model=qat_quantized_jit_model, device=cpu_device,
                                                                   input_size=(1, 3, 32, 32), num_samples=100)

    print("FP32 evaluation accuracy: {:.3f}".format(fp32_eval_accuracy))
    if static_quantization == True:
        print("*** Result of Static Quantization ***")
        print("INT8 evaluation accuracy: {:.3f}".format(static_int8_eval_accuracy))
    if quantization_aware_training == True:
        print("*** Result of Quantization Aware Training ***")
        print("INT8 evaluation accuracy: {:.3f}".format(qat_int8_eval_accuracy))
    print("")

    fp32_cpu_inference_latency = measure_inference_latency(model=model, device=cpu_device, input_size=(1, 3, 32, 32),
                                                           num_samples=100)
    fp32_gpu_inference_latency = measure_inference_latency(model=model, device=cuda_device,
                                                               input_size=(1, 3, 32, 32), num_samples=100)
    print("FP32 CPU Inference Latency: {:.2f} ms / sample".format(fp32_cpu_inference_latency * 1000))
    print("FP32 CUDA Inference Latency: {:.2f} ms / sample".format(fp32_gpu_inference_latency * 1000))

    if static_quantization == True:
        print("*** Result of Static Quantization ***")
        print("INT8 CPU Inference Latency: {:.2f} ms / sample".format(static_int8_cpu_inference_latency * 1000))
        print("INT8 JIT CPU Inference Latency: {:.2f} ms / sample".format(static_int8_jit_cpu_inference_latency * 1000))

        run_benchmark(static_jit_model_filepath, test_loader)
        run_benchmark(static_quantized_model_filepath, test_loader)
    if quantization_aware_training == True:
        print("*** Result of Quantization Aware Training ***")
        print("INT8 CPU Inference Latency: {:.2f} ms / sample".format(qat_int8_cpu_inference_latency * 1000))
        print("INT8 JIT CPU Inference Latency: {:.2f} ms / sample".format(qat_int8_jit_cpu_inference_latency * 1000))


def cpu():
    model_dir = "saved_models"
    model_filename = "resnet18_cifar10.pt"
    static_quantized_model_filename = "resnet18_quantized_cifar10.pt"
    model_filepath = os.path.join(model_dir, model_filename)
    static_quantized_model_filepath = os.path.join(model_dir, static_quantized_model_filename)

    cpu_device = torch.device("cpu:0")

    _, test_loader = prepare_dataloader(num_workers=8, train_batch_size=128, eval_batch_size=256)

    # Load quantized model.
    static_quantized_jit_model = load_torchscript_model(model_filepath=static_quantized_model_filepath,
                                                        device=cpu_device)

    _, int8_eval_accuracy = evaluate_model(model=static_quantized_jit_model, test_loader=test_loader, device=cpu_device,
                                           criterion=None)

    # Skip this assertion since the values might deviate a lot.
    # assert model_equivalence(model_1=model, model_2=quantized_jit_model, device=cpu_device, rtol=1e-01, atol=1e-02, num_tests=100, input_size=(1,3,32,32)), "Quantized model deviates from the original model too much!"

    print("INT8 evaluation accuracy: {:.3f}".format(int8_eval_accuracy))

    int8_jit_cpu_inference_latency = measure_inference_latency(model=static_quantized_jit_model, device=cpu_device,
                                                               input_size=(1, 3, 32, 32), num_samples=100)

    print("INT8 JIT CPU Inference Latency: {:.2f} ms / sample".format(int8_jit_cpu_inference_latency * 1000))

In [ ]:
# Tutorial Quantization
gpu(False, False, True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: -1 Eval Loss: 6.388 Eval Acc: 0.099
Epoch: 000 Train Loss: 4.341 Train Acc: 0.116 Eval Loss: 3.877 Eval Acc: 0.150
Epoch: 001 Train Loss: 2.140 Train Acc: 0.196 Eval Loss: 2.447 Eval Acc: 0.230
Epoch: 002 Train Loss: 1.913 Train Acc: 0.270 Eval Loss: 2.649 Eval Acc: 0.334
Epoch: 003 Train Loss: 1.765 Train Acc: 0.336 Eval Loss: 2.680 Eval Acc: 0.374
Epoch: 004 Train Loss: 1.653 Train Acc: 0.387 Eval Loss: 2.439 Eval Acc: 0.439
Epoch: 005 Train Loss: 1.567 Train Acc: 0.420 Eval Loss: 2.462 Eval Acc: 0.433
Epoch: 006 Train Loss: 1.496 Train Acc: 0.451 Eval Loss: 1.713 Eval Acc: 0.487
Epoch: 007 Train Loss: 1.427 Train Acc: 0.477 Eval Loss: 1.975 Eval Acc: 0.502
Epoch: 008 Train Loss: 1.378 Train Acc: 0.500 Eval Loss: 1.985 Eval Acc: 0.522
Epoch: 009 Train Loss: 1.321 Train Acc: 0.523 Eval Loss: 2.247 Eval Acc: 0.550
Epoch: 010 Train Loss: 1.268 Train Acc: 0.543 Eval Loss: 3.746 Eval Acc: 0.560
Epoch: 011 Train Loss: 1.222 Train Acc: 0.561 Eval Loss: 7.988 Eval Acc: 0.579
Epoch: 01

/usr/local/lib/python3.7/dist-packages/torch/quantization/observer.py:123: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."


Epoch: -1 Eval Loss: 0.624 Eval Acc: 0.792
Epoch: 000 Train Loss: 0.460 Train Acc: 0.839 Eval Loss: 0.532 Eval Acc: 0.819
Epoch: 001 Train Loss: 0.428 Train Acc: 0.850 Eval Loss: 0.517 Eval Acc: 0.823
Epoch: 002 Train Loss: 0.410 Train Acc: 0.857 Eval Loss: 0.513 Eval Acc: 0.827
Epoch: 003 Train Loss: 0.401 Train Acc: 0.861 Eval Loss: 0.507 Eval Acc: 0.828
Epoch: 004 Train Loss: 0.391 Train Acc: 0.863 Eval Loss: 0.502 Eval Acc: 0.830
Epoch: 005 Train Loss: 0.381 Train Acc: 0.867 Eval Loss: 0.502 Eval Acc: 0.830
Epoch: 006 Train Loss: 0.380 Train Acc: 0.866 Eval Loss: 0.500 Eval Acc: 0.833
Epoch: 007 Train Loss: 0.374 Train Acc: 0.868 Eval Loss: 0.499 Eval Acc: 0.831
Epoch: 008 Train Loss: 0.368 Train Acc: 0.871 Eval Loss: 0.497 Eval Acc: 0.832
Epoch: 009 Train Loss: 0.367 Train Acc: 0.871 Eval Loss: 0.494 Eval Acc: 0.833
Epoch: 010 Train Loss: 0.362 Train Acc: 0.875 Eval Loss: 0.492 Eval Acc: 0.834
Epoch: 011 Train Loss: 0.362 Train Acc: 0.873 Eval Loss: 0.492 Eval Acc: 0.836
Epoch: 01